In [1]:
import jieba
import numpy as np 
import pandas as pd 

dfapp = pd.read_csv('zz_apps_info_22k',sep = '\001',encoding = 'utf-8')
app_name = dfapp['app_name'].values.tolist()

jieba.load_userdict(app_name)

app_info = dfapp['app_info'].values.tolist()

app_info_segs = [jieba.lcut(x) for x in app_info]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\liangyun\AppData\Local\Temp\jieba.cache
Loading model cost 0.817 seconds.
Prefix dict has been built succesfully.


In [ ]:
import pkuseg

seg = pkuseg.pkuseg() 

app_info_segs = [seg.cut(x) for x in app_info]

In [2]:
import gensim
import logging
from gensim import corpora, models, similarities

#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
dictionary = corpora.Dictionary(app_info_segs)
dictionary.save('app_info.dict')  # store the dictionary

# 建立完整字典
dictionary.token2id

C:\ProgrameFiles\Anaconda\envs\py3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


{' ': 0,
 '-': 1,
 '.': 2,
 '/': 3,
 '0360': 4,
 '1': 5,
 '10': 6,
 '1010': 7,
 '11': 8,
 '2': 9,
 '2011': 10,
 '2017': 11,
 '3': 12,
 '360': 13,
 '4008': 14,
 '6%': 15,
 '905': 16,
 ':': 17,
 'D': 18,
 'com': 19,
 'http': 20,
 'm': 21,
 'rong360': 22,
 '—': 23,
 '、': 24,
 '。': 25,
 '【': 26,
 '】': 27,
 '一天': 28,
 '上市': 29,
 '下述': 30,
 '不': 31,
 '不停': 32,
 '专家': 33,
 '专属': 34,
 '业务范围': 35,
 '个': 36,
 '个人': 37,
 '个人消费': 38,
 '中国': 39,
 '为': 40,
 '为什么': 41,
 '于': 42,
 '产品': 43,
 '亿美元': 44,
 '仅': 45,
 '介绍': 46,
 '企业': 47,
 '优势': 48,
 '优惠': 49,
 '优质': 50,
 '估值': 51,
 '低至': 52,
 '使用': 53,
 '便捷': 54,
 '信用卡': 55,
 '信赖': 56,
 '俱乐部': 57,
 '值得': 58,
 '免费': 59,
 '全面': 60,
 '全面解决': 61,
 '公众': 62,
 '公司': 63,
 '内容': 64,
 '出行': 65,
 '划算': 66,
 '利率': 67,
 '办卡': 68,
 '即可': 69,
 '号': 70,
 '吃喝玩乐': 71,
 '各种': 72,
 '各行': 73,
 '吹牛': 74,
 '和': 75,
 '在': 76,
 '垂直': 77,
 '大全': 78,
 '奉献': 79,
 '学习': 80,
 '安全': 81,
 '完成': 82,
 '官方': 83,
 '官方网站': 84,
 '实时': 85,
 '客服': 86,
 '小白': 87,
 '已': 88,
 '平台': 89,
 '年': 90,


In [3]:
len(dictionary)

117107

In [4]:
# 重点词典，去除文档频率2次以下和0.9以上的词
dictionary.filter_extremes(no_below=2,no_above=0.9,keep_n = 100000,keep_tokens = None)
dictionary.compactify()
#dictionary.token2id

In [5]:
len(dictionary.token2id)

49030

In [6]:
dictionary.token2id

{' ': 0,
 '-': 1,
 '.': 2,
 '/': 3,
 '0360': 4,
 '1': 5,
 '10': 6,
 '1010': 7,
 '11': 8,
 '2': 9,
 '2011': 10,
 '2017': 11,
 '3': 12,
 '360': 13,
 '4008': 14,
 '6%': 15,
 '905': 16,
 ':': 17,
 'D': 18,
 'com': 19,
 'http': 20,
 'm': 21,
 '—': 22,
 '、': 23,
 '。': 24,
 '【': 25,
 '】': 26,
 '一天': 27,
 '上市': 28,
 '下述': 29,
 '不': 30,
 '不停': 31,
 '专家': 32,
 '专属': 33,
 '业务范围': 34,
 '个': 35,
 '个人': 36,
 '个人消费': 37,
 '中国': 38,
 '为': 39,
 '为什么': 40,
 '于': 41,
 '产品': 42,
 '亿美元': 43,
 '仅': 44,
 '介绍': 45,
 '企业': 46,
 '优势': 47,
 '优惠': 48,
 '优质': 49,
 '估值': 50,
 '低至': 51,
 '使用': 52,
 '便捷': 53,
 '信用卡': 54,
 '信赖': 55,
 '俱乐部': 56,
 '值得': 57,
 '免费': 58,
 '全面': 59,
 '全面解决': 60,
 '公众': 61,
 '公司': 62,
 '内容': 63,
 '出行': 64,
 '划算': 65,
 '利率': 66,
 '办卡': 67,
 '即可': 68,
 '号': 69,
 '吃喝玩乐': 70,
 '各种': 71,
 '各行': 72,
 '吹牛': 73,
 '和': 74,
 '在': 75,
 '垂直': 76,
 '大全': 77,
 '奉献': 78,
 '学习': 79,
 '安全': 80,
 '完成': 81,
 '官方': 82,
 '官方网站': 83,
 '实时': 84,
 '客服': 85,
 '小白': 86,
 '已': 87,
 '平台': 88,
 '年': 89,
 '年化': 90,
 '微信'

### 训练词袋模型

In [9]:
corpus = [dictionary.doc2bow(text) for text in app_info_segs]

In [10]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

2019-01-24 17:37:05,386 : INFO : collecting document frequencies
2019-01-24 17:37:05,391 : INFO : PROGRESS: processing document #0
2019-01-24 17:37:05,668 : INFO : PROGRESS: processing document #10000
2019-01-24 17:37:05,883 : INFO : PROGRESS: processing document #20000
2019-01-24 17:37:05,947 : INFO : calculating IDF weights for 22262 documents and 49029 features (1905366 matrix non-zeros)


### 训练lda模型

In [7]:
import logging
import os
from gensim import corpora, models, similarities

In [8]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [12]:
lda = models.LdaModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=50, iterations=100, passes=100)

2019-01-24 17:37:35,398 : INFO : using symmetric alpha at 0.02
2019-01-24 17:37:35,400 : INFO : using symmetric eta at 0.02
2019-01-24 17:37:35,422 : INFO : using serial LDA version on this node
2019-01-24 17:37:35,768 : INFO : running online (multi-pass) LDA training, 50 topics, 100 passes over the supplied corpus of 22262 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 100x with a convergence threshold of 0.001000
2019-01-24 17:37:36,757 : INFO : PROGRESS: pass 0, at document #2000/22262
2019-01-24 17:37:37,757 : INFO : merging changes from 2000 documents into a model of 22262 documents
2019-01-24 17:37:38,116 : INFO : topic #13 (0.020): 0.004*"贷款" + 0.004*"、" + 0.004*"借款" + 0.003*"-" + 0.003*" " + 0.003*"【" + 0.003*"】" + 0.003*"：" + 0.003*"；" + 0.002*"！"
2019-01-24 17:37:38,121 : INFO : topic #35 (0.020): 0.005*"=" + 0.004*" " + 0.004*"；" + 0.003*"-" + 0.003*"、" + 0.003*"清理" + 0.003*"！" + 0.003*"：" + 0.003*"信用" + 0.003*"*"


2019-01-24 17:37:52,544 : INFO : topic diff=0.526154, rho=0.377964
2019-01-24 17:37:53,368 : INFO : PROGRESS: pass 0, at document #16000/22262
2019-01-24 17:37:54,313 : INFO : merging changes from 2000 documents into a model of 22262 documents
2019-01-24 17:37:54,669 : INFO : topic #26 (0.020): 0.018*" " + 0.007*"保险" + 0.006*"检验" + 0.004*"初学者" + 0.004*"保险产品" + 0.003*"伴侣" + 0.003*"出单" + 0.003*"逐渐" + 0.003*"动图" + 0.003*"签单"
2019-01-24 17:37:54,671 : INFO : topic #44 (0.020): 0.005*"转账" + 0.005*"恢复" + 0.005*"密码" + 0.004*"解谜" + 0.004*"银行" + 0.004*"全景" + 0.004*"手机" + 0.004*"院校" + 0.003*"题材" + 0.003*"手机用户"
2019-01-24 17:37:54,673 : INFO : topic #10 (0.020): 0.010*" " + 0.007*"," + 0.006*"." + 0.006*"-" + 0.005*"=" + 0.005*"、" + 0.004*"你" + 0.004*"：" + 0.004*"可以" + 0.003*"/"
2019-01-24 17:37:54,676 : INFO : topic #7 (0.020): 0.015*"　" + 0.010*"开奖" + 0.006*"导购" + 0.005*"更省" + 0.005*"美观" + 0.005*"彩种" + 0.004*"开奖号码" + 0.004*"功能强大" + 0.004*"路由器" + 0.004*"界面"
2019-01-24 17:37:54,678 : INFO : topic

2019-01-24 17:38:08,246 : INFO : topic diff=0.194853, rho=0.275963
2019-01-24 17:38:09,118 : INFO : PROGRESS: pass 1, at document #6000/22262
2019-01-24 17:38:10,011 : INFO : merging changes from 2000 documents into a model of 22262 documents
2019-01-24 17:38:10,360 : INFO : topic #3 (0.020): 0.007*"装修" + 0.006*"历史数据" + 0.005*"．" + 0.005*"相册" + 0.005*"参数" + 0.004*"会计" + 0.004*"题" + 0.004*"考点" + 0.004*"缩小" + 0.004*"歌曲"
2019-01-24 17:38:10,362 : INFO : topic #48 (0.020): 0.028*"宝宝" + 0.015*"巴士" + 0.010*"捕鱼" + 0.007*"早教" + 0.007*"babybus" + 0.006*"BOSS" + 0.005*"街机" + 0.005*"孩子" + 0.005*"启蒙" + 0.004*"_________"
2019-01-24 17:38:10,364 : INFO : topic #39 (0.020): 0.010*"小说" + 0.005*"钓鱼" + 0.004*"小朋友" + 0.004*"绘画" + 0.003*"随借" + 0.003*"读者" + 0.003*"随" + 0.003*"武侠" + 0.003*" " + 0.003*"都市"
2019-01-24 17:38:10,366 : INFO : topic #4 (0.020): 0.007*"☆" + 0.006*"洗车" + 0.006*"链" + 0.006*"区块" + 0.006*"汉字" + 0.005*"车主" + 0.004*"卡牌" + 0.004*"幼儿园" + 0.004*"办事" + 0.003*"迅捷"
2019-01-24 17:38:10,368 : I

2019-01-24 17:38:25,796 : INFO : PROGRESS: pass 1, at document #20000/22262
2019-01-24 17:38:26,662 : INFO : merging changes from 2000 documents into a model of 22262 documents
2019-01-24 17:38:27,030 : INFO : topic #39 (0.020): 0.009*"小朋友" + 0.006*"小说" + 0.005*"钓鱼" + 0.005*"读者" + 0.005*"绘画" + 0.004*"章节" + 0.003*"写作" + 0.003*"武侠" + 0.003*"言情" + 0.003*"玄幻"
2019-01-24 17:38:27,032 : INFO : topic #1 (0.020): 0.009*"赚钱" + 0.007*"红包" + 0.007*"麻将" + 0.006*"任务" + 0.006*"奖励" + 0.005*"赚" + 0.004*"！" + 0.004*"金币" + 0.004*"邀请" + 0.004*"斗地主"
2019-01-24 17:38:27,033 : INFO : topic #15 (0.020): 0.049*"\" + 0.011*""" + 0.011*"律师" + 0.009*"遗漏" + 0.006*"法律" + 0.006*"塔防" + 0.005*"冷热" + 0.005*"到位" + 0.004*"战士" + 0.004*"缺少"
2019-01-24 17:38:27,035 : INFO : topic #37 (0.020): 0.011*"报警" + 0.007*"行车" + 0.007*"行业资讯" + 0.007*"种植" + 0.007*"医学" + 0.007*"防盗" + 0.005*"检查" + 0.005*"养殖" + 0.005*"生产" + 0.005*"加油站"
2019-01-24 17:38:27,037 : INFO : topic #42 (0.020): 0.016*"★" + 0.009*"证券" + 0.008*"行情" + 0.008*"开户" + 

2019-01-24 17:38:40,854 : INFO : merging changes from 2000 documents into a model of 22262 documents
2019-01-24 17:38:41,220 : INFO : topic #31 (0.020): 0.008*"多开" + 0.008*"车载" + 0.006*"＊" + 0.006*"双开" + 0.005*"趣闻" + 0.005*"深度" + 0.005*"命中" + 0.005*"图案" + 0.005*"行车记录仪" + 0.005*"个人用户"
2019-01-24 17:38:41,222 : INFO : topic #38 (0.020): 0.006*"违法" + 0.005*"题目" + 0.005*"驾驶" + 0.005*"单机游戏" + 0.004*"跨服" + 0.004*"游客" + 0.003*"智力" + 0.003*"计时" + 0.003*"轻巧" + 0.003*"大牛"
2019-01-24 17:38:41,224 : INFO : topic #27 (0.020): 0.008*"代练" + 0.007*"往期" + 0.005*"▲" + 0.005*"变声" + 0.005*"攻击" + 0.004*"外出" + 0.004*"园艺" + 0.004*"可信" + 0.003*"主导" + 0.003*"蓝光"
2019-01-24 17:38:41,225 : INFO : topic #45 (0.020): 0.021*"游戏" + 0.012*"玩法" + 0.011*"玩家" + 0.010*"《" + 0.010*"》" + 0.009*"手游" + 0.008*"！" + 0.007*"战斗" + 0.006*"经典" + 0.006*"画面"
2019-01-24 17:38:41,227 : INFO : topic #9 (0.020): 0.012*"家长" + 0.009*"作业" + 0.008*"学校" + 0.007*"孩子" + 0.007*"学生" + 0.006*"教师" + 0.006*"老师" + 0.005*"家校" + 0.005*"班级" + 0.005*"工具

...

2019-01-24 18:16:01,288 : INFO : topic #17 (0.020): 0.010*"收款" + 0.007*"壁纸" + 0.005*"桌面" + 0.005*"应用程序" + 0.005*"动物" + 0.005*"照片" + 0.004*"微商" + 0.004*"主题" + 0.004*"监控" + 0.004*"是否"
2019-01-24 18:16:01,289 : INFO : topic #21 (0.020): 0.045*" " + 0.008*"健身" + 0.007*"the" + 0.007*"and" + 0.007*"to" + 0.006*"," + 0.006*"运动" + 0.005*"you" + 0.004*"顾客" + 0.004*"跑步"
2019-01-24 18:16:01,291 : INFO : topic #19 (0.020): 0.007*"驾校" + 0.007*"学车" + 0.006*"复习" + 0.005*"教练" + 0.004*"工资" + 0.004*"药店" + 0.004*"学员" + 0.004*"加班" + 0.004*"备考" + 0.004*"摩托车"
2019-01-24 18:16:01,292 : INFO : topic #29 (0.020): 0.010*"社保" + 0.010*"计算" + 0.008*"计算器" + 0.008*"公积金" + 0.006*"参保" + 0.005*"便民服务" + 0.004*"采购" + 0.004*"等额" + 0.004*"坐骑" + 0.004*"直销银行"
2019-01-24 18:16:01,296 : INFO : topic #5 (0.020): 0.008*"政务" + 0.005*"人才" + 0.005*"一边" + 0.005*"象棋" + 0.004*"物理" + 0.004*"代替" + 0.004*"得分" + 0.004*"概念" + 0.004*"赛车" + 0.004*"违反"
2019-01-24 18:16:01,303 : INFO : topic diff=0.011695, rho=0.095725
2019-01-24 18:16:02,068 

2019-01-24 18:16:14,324 : INFO : merging changes from 2000 documents into a model of 22262 documents
2019-01-24 18:16:14,654 : INFO : topic #14 (0.020): 0.006*"维修" + 0.005*"广播" + 0.004*"物业" + 0.004*"上网" + 0.003*"电视" + 0.003*"诚信" + 0.003*"洗衣" + 0.003*"新闻" + 0.003*"传输" + 0.003*"综合性"
2019-01-24 18:16:14,655 : INFO : topic #25 (0.020): 0.014*"消除" + 0.007*"签名" + 0.006*"架子鼓" + 0.006*"抢红包" + 0.006*"湖南" + 0.005*"想象力" + 0.005*"钢琴" + 0.005*"关卡" + 0.004*"体温" + 0.004*"模拟器"
2019-01-24 18:16:14,657 : INFO : topic #28 (0.020): 0.010*"食物" + 0.010*"就诊" + 0.009*"公主" + 0.006*"减肥" + 0.005*"端游" + 0.005*"换装" + 0.005*"门诊" + 0.004*"收藏品" + 0.004*"动画片" + 0.004*"睡觉"
2019-01-24 18:16:14,660 : INFO : topic #35 (0.020): 0.016*"经纪人" + 0.010*"清理" + 0.008*"房地产" + 0.005*"读取" + 0.005*"垃圾" + 0.004*"加速" + 0.004*"新动向" + 0.004*"报备" + 0.004*"手电筒" + 0.003*"闪光灯"
2019-01-24 18:16:14,662 : INFO : topic #33 (0.020): 0.012*"存管" + 0.009*"◆" + 0.006*"合规" + 0.006*"娃娃" + 0.006*"三国" + 0.006*"风控" + 0.005*"新手" + 0.005*"资产" + 0.005*"投" + 

2019-01-24 18:16:27,919 : INFO : topic #31 (0.020): 0.008*"车载" + 0.008*"多开" + 0.006*"十分" + 0.005*"＊" + 0.005*"个人用户" + 0.005*"双开" + 0.005*"趣闻" + 0.005*"综合类" + 0.005*"分身" + 0.005*"图案"
2019-01-24 18:16:27,920 : INFO : topic #0 (0.020): 0.008*"字体" + 0.007*"小说" + 0.007*"VR" + 0.005*"中医" + 0.004*"图书" + 0.004*"书籍" + 0.003*"电影" + 0.003*"执业" + 0.003*"好吃" + 0.003*"影片"
2019-01-24 18:16:27,923 : INFO : topic #32 (0.020): 0.010*"员工" + 0.006*"库存" + 0.006*"餐厅" + 0.006*"考勤" + 0.006*"飞机" + 0.005*"协作" + 0.005*"杀" + 0.004*"热点资讯" + 0.004*"公益" + 0.004*"日程"
2019-01-24 18:16:27,925 : INFO : topic #24 (0.020): 0.007*"开门" + 0.004*"关" + 0.004*"种子" + 0.004*"网络电话" + 0.003*"爸妈" + 0.003*"山西" + 0.003*"娱乐性" + 0.003*"装逼" + 0.003*"多角度" + 0.003*"汽配"
2019-01-24 18:16:27,932 : INFO : topic diff=0.011459, rho=0.095290
2019-01-24 18:16:28,667 : INFO : PROGRESS: pass 98, at document #18000/22262
2019-01-24 18:16:29,352 : INFO : merging changes from 2000 documents into a model of 22262 documents
2019-01-24 18:16:29,694 : INFO

2019-01-24 18:16:41,976 : INFO : topic #12 (0.020): 0.023*"市民" + 0.010*"收银" + 0.009*"购彩" + 0.009*"公共服务" + 0.006*"从业人员" + 0.005*"广西" + 0.005*"人力资源" + 0.005*"使用者" + 0.004*"智能手表" + 0.004*"重庆时时彩"
2019-01-24 18:16:41,977 : INFO : topic #47 (0.020): 0.008*"快速借钱" + 0.007*"工薪族" + 0.006*"蓝领" + 0.006*"缺钱" + 0.006*"拍卖" + 0.005*"实体店" + 0.005*"信用钱包" + 0.005*"文玩" + 0.005*"企业主" + 0.004*"提额"
2019-01-24 18:16:41,979 : INFO : topic #44 (0.020): 0.008*"转账" + 0.007*"WiFi" + 0.006*"工作效率" + 0.006*"汇款" + 0.006*"恢复" + 0.005*"全景" + 0.005*"帐号" + 0.005*"题材" + 0.005*"NFC" + 0.004*"征信报告"
2019-01-24 18:16:41,980 : INFO : topic #26 (0.020): 0.024*" " + 0.008*"办卡" + 0.005*"检验" + 0.005*"轨道" + 0.004*"初学者" + 0.004*"逐渐" + 0.004*"4S店" + 0.003*"交通银行" + 0.003*"广发" + 0.003*"功能齐全"
2019-01-24 18:16:41,988 : INFO : topic diff=0.012026, rho=0.094860
2019-01-24 18:16:42,800 : INFO : PROGRESS: pass 99, at document #8000/22262
2019-01-24 18:16:43,521 : INFO : merging changes from 2000 documents into a model of 22262 documents
2019-

2019-01-24 18:16:56,389 : INFO : topic #37 (0.020): 0.010*"报警" + 0.008*"行业资讯" + 0.007*"行车" + 0.007*"医学" + 0.006*"生产" + 0.006*"种植" + 0.005*"疾病" + 0.005*"防盗" + 0.005*"检查" + 0.005*"体检"
2019-01-24 18:16:56,391 : INFO : topic #42 (0.020): 0.015*"★" + 0.010*"行情" + 0.009*"证券" + 0.009*"股票" + 0.008*"炒股" + 0.008*"开户" + 0.006*"敌人" + 0.005*"基金" + 0.004*"财经" + 0.004*"港股"
2019-01-24 18:16:56,393 : INFO : topic #43 (0.020): 0.007*"运势" + 0.007*"星座" + 0.007*"八字" + 0.005*"志愿" + 0.004*"黄历" + 0.004*"儿歌" + 0.004*"农历" + 0.004*"日历" + 0.004*"志愿者" + 0.003*"起名"
2019-01-24 18:16:56,395 : INFO : topic #22 (0.020): 0.006*"摄影" + 0.006*"房产" + 0.004*"个股" + 0.004*"同城" + 0.003*"投诉" + 0.003*"聚会" + 0.003*"送达" + 0.003*"听说" + 0.003*"全城" + 0.003*"上面"
2019-01-24 18:16:56,403 : INFO : topic diff=0.012052, rho=0.094860
2019-01-24 18:16:57,140 : INFO : PROGRESS: pass 99, at document #22000/22262
2019-01-24 18:16:57,868 : INFO : merging changes from 2000 documents into a model of 22262 documents
2019-01-24 18:16:58,208 : INFO : 

In [13]:
from gensim import matutils

features  = matutils.corpus2dense(list(lda[corpus_tfidf]),num_terms= 50).T

In [17]:
dfea = pd.DataFrame(features)

In [21]:
lda.save('lda.model')

2019-01-24 19:34:20,252 : INFO : saving LdaState object under lda.model.state, separately None
2019-01-24 19:34:20,361 : INFO : saved lda.model.state
2019-01-24 19:34:20,406 : INFO : saving LdaModel object under lda.model, separately ['expElogbeta', 'sstats']
2019-01-24 19:34:20,409 : INFO : storing np array 'expElogbeta' to lda.model.expElogbeta.npy
2019-01-24 19:34:20,419 : INFO : not storing attribute state
2019-01-24 19:34:20,422 : INFO : not storing attribute id2word
2019-01-24 19:34:20,424 : INFO : not storing attribute dispatcher
2019-01-24 19:34:20,431 : INFO : saved lda.model


In [22]:
topics = lda.print_topics()

2019-01-24 19:35:09,838 : INFO : topic #33 (0.020): 0.013*"存管" + 0.008*"◆" + 0.006*"娃娃" + 0.006*"三国" + 0.006*"合规" + 0.006*"风控" + 0.006*"新手" + 0.006*"资产" + 0.005*"投" + 0.005*"年化"
2019-01-24 19:35:09,840 : INFO : topic #41 (0.020): 0.008*"、" + 0.008*"服务" + 0.005*"信息" + 0.005*"提供" + 0.005*"查询" + 0.005*"平台" + 0.005*"为" + 0.004*"互联网" + 0.004*"管理" + 0.004*"用户"
2019-01-24 19:35:09,842 : INFO : topic #3 (0.020): 0.010*"装修" + 0.008*"历史数据" + 0.007*"考点" + 0.007*"配送" + 0.005*"题" + 0.005*"参数" + 0.005*"婚礼" + 0.004*"．" + 0.004*"歌曲" + 0.004*"婴儿"
2019-01-24 19:35:09,844 : INFO : topic #45 (0.020): 0.019*"游戏" + 0.011*"玩法" + 0.010*"玩家" + 0.010*"》" + 0.010*"《" + 0.009*"手游" + 0.009*"战斗" + 0.007*"画面" + 0.006*"挑战" + 0.006*"3D"
2019-01-24 19:35:09,846 : INFO : topic #24 (0.020): 0.010*"开门" + 0.006*"Synology" + 0.005*"装逼" + 0.004*"老婆" + 0.004*"相宜" + 0.004*"爸妈" + 0.004*"网络电话" + 0.004*"关" + 0.003*"l" + 0.003*"娱乐性"
2019-01-24 19:35:09,847 : INFO : topic #44 (0.020): 0.007*"工作效率" + 0.007*"WiFi" + 0.007*"全景" + 0.00

In [25]:
topics = lda.print_topics(num_topics=50)

2019-01-24 19:35:54,587 : INFO : topic #0 (0.020): 0.010*"字体" + 0.008*"VR" + 0.007*"小说" + 0.006*"中医" + 0.004*"好吃" + 0.004*"治疗" + 0.004*"图书" + 0.003*"执业" + 0.003*"书籍" + 0.003*"电影"
2019-01-24 19:35:54,591 : INFO : topic #1 (0.020): 0.006*"滤镜" + 0.006*"照片" + 0.006*"拼图" + 0.005*"麻将" + 0.004*"赚" + 0.004*"制作" + 0.004*"拍摄" + 0.004*"特效" + 0.004*"贴纸" + 0.003*"邀请"
2019-01-24 19:35:54,593 : INFO : topic #2 (0.020): 0.051*"开奖" + 0.025*"走势" + 0.022*"中奖" + 0.021*"彩种" + 0.021*"分析" + 0.018*"预测" + 0.013*"彩票" + 0.010*"结果" + 0.009*"专家" + 0.008*"选"
2019-01-24 19:35:54,596 : INFO : topic #3 (0.020): 0.010*"装修" + 0.008*"历史数据" + 0.007*"考点" + 0.007*"配送" + 0.005*"题" + 0.005*"参数" + 0.005*"婚礼" + 0.004*"．" + 0.004*"歌曲" + 0.004*"婴儿"
2019-01-24 19:35:54,598 : INFO : topic #4 (0.020): 0.008*"链" + 0.008*"汉字" + 0.007*"区块" + 0.007*"洗车" + 0.007*"☆" + 0.006*"卡牌" + 0.005*"高手" + 0.005*"家园" + 0.005*"币" + 0.004*"数字"
2019-01-24 19:35:54,600 : INFO : topic #5 (0.020): 0.011*"政务" + 0.006*"物理" + 0.006*"人才" + 0.005*"茶" + 0.005*"象

2019-01-24 19:35:54,674 : INFO : topic #46 (0.020): 0.007*"约会" + 0.006*"小孩" + 0.005*"角色" + 0.005*"农业" + 0.005*"交友" + 0.004*"关卡" + 0.004*"★" + 0.004*"加工" + 0.004*"所以" + 0.003*"武器"
2019-01-24 19:35:54,676 : INFO : topic #47 (0.020): 0.008*"快速借钱" + 0.007*"实体店" + 0.006*"拍卖" + 0.005*"文玩" + 0.005*"缺钱" + 0.005*"提额" + 0.005*"工薪族" + 0.005*"蓝领" + 0.004*"宅" + 0.004*"铃音"
2019-01-24 19:35:54,678 : INFO : topic #48 (0.020): 0.024*"宝宝" + 0.017*"捕鱼" + 0.009*"巴士" + 0.008*"BOSS" + 0.008*"街机" + 0.007*"早教" + 0.006*"幼儿" + 0.006*"启蒙" + 0.005*"RPG" + 0.005*"坦克"
2019-01-24 19:35:54,679 : INFO : topic #49 (0.020): 0.025*"学习" + 0.017*"考试" + 0.013*"英语" + 0.013*"课程" + 0.012*"练习" + 0.010*"题库" + 0.009*"单词" + 0.008*"名师" + 0.007*"科目" + 0.006*"学"


In [29]:
import json 
#json.dump(topics) 
with open('lda_topics','w') as f:
    json.dump(topics,f)

In [31]:
dfea.insert(0,'app',dfapp['app_name'])

In [33]:
dfea.columns = ['app'] + ['topic_'+str(i) for i in range(1,51)]

In [85]:
dfea.to_csv('app_lda.csv',sep ='\t',encoding = 'utf-8',index = None)

### 计算词聚类

In [27]:
df.columns = ['word'] + ['vec_'+str(i) for i in range(1,101)]

In [36]:
X = dfea.iloc[:,1:].values

In [37]:
X.shape

(22262, 50)

In [38]:
from sklearn.cluster import KMeans

In [39]:
clt = KMeans(n_clusters = 100)

In [42]:
app_clusters = clt.fit_predict(X)

In [43]:
len(app_clusters)

22262

In [44]:
app_clusters

array([58, 52, 82, ..., 26, 45, 85])

In [46]:
dfcluster = dfea[['app']].copy()

In [48]:
dfcluster['app_lda_cluster'] = app_clusters.tolist()

In [50]:
dfcluster

,app,app_lda_cluster
0,融360,58
1,微信,52
2,设置,82
3,支付宝,79
4,信息,89
5,QQ,89
6,指南针,29
7,手机淘宝,11
8,云服务,92
9,WiFi万能钥匙,40


In [52]:
dfgroup = dfcluster.groupby('app_lda_cluster').agg({'app':lambda l:','.join([x for x in l])})

In [73]:
print(dfgroup.loc[16].values)

['电信营业厅,永安行,阿里TV助手,ETC车宝,芒果银行,360手机急救箱,闪租侠,和信贷,凤凰金融,蘑菇丁,身份宝,机蜜,有机转转,粤通卡,鑫格理财,探索果智能遥控,乐丰服务中心,钱盆网,天津农商银行,掌悦理财,圣贤财富,多融理财,e钱包,迪信通,多多理财,微家,领奇理财,狐狸慧赚,钱路理财,友金所,秋田财富,习讯云,云服务平台,得宝理财,乐金所,乾包,百金贷,赚吧理财,爱木财富,兔司机,普汇云通理财,众邦直销银行,联银普惠,猴子理财,牛拜单车,短融网理财,北京PK10大亨计划,高架养护,分享加,格力智联,合时代金融,积财金融,车镇车源,包公有财,银狐财富,发票助手,芒果金融,盈鱼理财,风车理财,钱时代理财,来浙投理财,诺米之家,小微时贷,司机宝,盎盎理财,票票喵理财,繁锦财富,智慧安监企安助手,UU理财,星时贷理财,道口贷,浙江ETC,爱换机店员版,前金融,和动力,星火钱包,泰然金融,拓道金服,网格金服,懒投资,惠农聚宝,壹佰金融,小猪理财,中网国投,三益宝,前海航交所,来转吧,好麦金融,余易贷理财,哎呦抓娃娃,蜜蜂有钱,民投金服,芝麻宝,宝点理财,一鼎金融,小鸡理财,徽盐金融,投米RA,鹏金所,车蚁金服,一桶金理财投资,乐享宝,梧桐诚选,钢票网,长久贷,海星宝理财,爱租机,采招圈,美客搜,掌盈金服,优信呗,爱大厨,极光金融,汇泰在线,小行家金服,联联单车,贵阳房产超市,信投宝,新华金典理财,西安发布,汇票助手,咔哇机抓娃娃,全日空海淘,筷来财,银海金服,网投网,房价管家,众金所,佬司机,囧羊理财,金蜂财富,固金所,信租,知合金服理财,阿尔法金融,信贷通,小猪罐子,板板金服,富通贷,中兴财富通,来啦,浙鼎金融,旧衣天下通,招财汇宝,巨潮金融,乐为金融,融资线,付霸,欢乐合家理财,龙龙理财,信用大师,钱宝财,百姓金融,富仁金融,嘉石榴,佳缘金融,空中金融,余薪乐,帮呗,乐实习,抢钱通,万家贷,城道理财,融客,网汇贷,来存吧理财,习讯宁夏,普汇聚财']


In [80]:
dfgroup.insert(0,'cluster',['cluster_'+str(i) for i in range(len(dfgroup))])

In [81]:
dfgroup

,cluster,app
app_lda_cluster,,
0,cluster_0,"工程模式,UC浏览器,动态壁纸选择器,迅雷,QQ空间,信息助手,我的文件,百度糯米,美篇,淘..."
1,cluster_1,"饿了么,口碑,魅族商城,服务指南,云集VIP,京东到家,和生活爱辽宁,和多号,铂涛旅行,借条..."
2,cluster_2,"平安普惠,现金借款,人人贷借款,360贷款导航,卡牛信用管家,优借,还呗,51人品贷,乐贷款..."
3,cluster_3,"中国移动,云闪付,和聚汇,融e购,和生活,摇钱树,安徽掌上10000,移动旗舰店,贵州石油,..."
4,cluster_4,"便签,百度地图,光大银行,企业微信,四川移动掌上营业厅,OneNote,蚂蚁财富,移动门户,..."
5,cluster_5,"腾讯课堂,中英互译,崩坏3,扇贝单词,Google Play 音乐,驾考宝典科目一,一起考教..."
6,cluster_6,"集结号捕鱼,小米枪战,楚留香,群友斗地主,欢乐切水果大作战,全民枪战,我在大清当皇帝,猎魂觉..."
7,cluster_7,"立刷,丰收互联,PP体育,萌虎白卡,国泰君安君弘,闪电白卡,蓝薪卡,得富宝,兰州银行,智能公..."
8,cluster_8,"智能解锁,Dark,SOS,西瓜商城,Retouch,Foundation,Discover..."


In [83]:
dfgroup.to_csv('app_lda_cluster.csv',sep = '\t',encoding = 'utf-8',index = None) 

### APP向量

In [56]:
word_clusters = [set(x.split(',')) for x in dfgroup['word']]

In [65]:
app_vecs = [[sum([1  for w in app_segs if w in cluster]) for cluster in word_clusters] for app_segs in app_info_segs]

In [67]:
len(app_vecs)

22262

In [69]:
import numpy as np 
import pandas as pd


In [71]:
dfappvec = pd.DataFrame(app_vecs,columns = ['cluster_' +str(i) for i in range(1,501)])

In [76]:
dfappvec.insert(0,'app',dfapp['app_name'])

In [87]:
dfappvec.to_csv('app_word_cluster_vec.csv',sep = '\t',encoding = 'utf-8',index = None)

In [84]:
dfapp.loc[9,'app_info']

'免费安全WiFi热点，无忧一键连接。无论身处何地，只要打开APP，看到分享热点，点击连接即可放心上网。 【免费】节省网络流量快人一步，随时随地免费连接，畅享24小时免费WiFi，再也不用担心月底流量不够用啦！ 【便捷】点击一键查询，秒速发现免费WiFi热点，一键安全连接，操作简单，上网就是这么轻松。内置中国移动CMCC、中国联通ChinaUnicom、中国电信ChinaNet等运营商热点，安全连接，方便便捷。【管理】全面掌握上网情况，网速测试，信号检测，WiFi好伴侣，WiFi好管家，上网神器。 【精彩】找工作，看直播，看新闻, 玩游戏——狼人杀，玩直播，看视频，看头条，网购，共享单车等更多便捷工具和精彩资讯，等你来探索。 【其他】随身WiFi功能，一键手机变热点；流量统计查看流量消耗。 ---更多实用贴心功能，敬请关注和期待---'

In [68]:
len(app_info_segs)

22262

In [86]:
dfgroup.loc[4,'word']

'故事,开启,开始,快乐,探索,瞬间,美好,美丽,陪伴,充满,尽情,之旅,时光,点滴,回忆,一段,旅程'

In [89]:
dfappword = pd.read_csv('app_word_cluster_vec.csv',sep = '\t',encoding = 'utf-8',index_col = 0)

In [93]:
app_word = dfappword.loc[['融360','微信'],:].sum(axis = 0)
app_word

cluster_1       0
cluster_2       5
cluster_3       0
cluster_4       5
cluster_5       1
cluster_6       0
cluster_7       1
cluster_8       0
cluster_9       1
cluster_10      1
cluster_11      0
cluster_12      0
cluster_13      3
cluster_14      2
cluster_15      1
cluster_16      0
cluster_17      0
cluster_18      0
cluster_19      0
cluster_20     11
cluster_21      0
cluster_22      0
cluster_23      0
cluster_24      1
cluster_25      0
cluster_26      0
cluster_27      0
cluster_28      1
cluster_29      0
cluster_30      1
               ..
cluster_471     0
cluster_472     0
cluster_473     0
cluster_474     0
cluster_475     0
cluster_476     0
cluster_477     0
cluster_478     0
cluster_479     0
cluster_480     0
cluster_481     0
cluster_482     4
cluster_483     0
cluster_484     0
cluster_485     1
cluster_486     0
cluster_487     0
cluster_488     0
cluster_489     1
cluster_490     1
cluster_491     0
cluster_492     0
cluster_493     0
cluster_494     0
cluster_49

In [273]:
dfgroup.columns = ['apps']

In [ ]:
dfgroup.to_csv('app_cluster',sep = '\t',encoding = 'utf-8')

### appcate

In [338]:
import numpy as np 
import pandas as pd 



In [344]:
dfapp = pd.read_csv('zz_apps_info_22k',sep = '\001',encoding = 'utf-8')
dfapp = dfapp.drop_duplicates(subset = ['app_name'])

In [345]:
dfcate = dfapp[['app_name','app_cate']].copy()

In [346]:
dfcategroup = dfcate.groupby('app_cate').agg({'app_name':lambda l:','.join([x for x in l])})

In [348]:
dfcategroup.insert(0,'app_cate',dfcategroup.index)

In [349]:
dfcategroup.insert(0,'cate_name')

,app_cate,app_name
app_cate,,
休闲益智,休闲益智,"珊瑚海,媒体存储器,贪吃蛇大作战,球球大作战,节奏大师,迷你世界,我的汤姆猫,腾讯桌球,我的..."
体育竞速,体育竞速,"登山赛车,天天飞车,城市飞车,街头篮球,FIFA足球世界,街机台球大师,疾风飞车世界,汤姆猫..."
健康,健康,"华为穿戴,三星健康,平安好医生,悦动圈,Keep,小米运动,咕咚,健康卫士,乐心运动,优健康..."
儿童,儿童,"儿童模式,宝宝树孕育,小伴龙,儿歌点点,西瓜皮,凯叔讲故事,贝乐虎儿歌,宝宝小警察,宝宝巴士..."
出行,出行,"高德地图,百度地图,驾驶模式,神州专车,位置服务,找回手机,微车违章查询,滴滴顺风车,奥维互..."
办公,办公,"WPS Office,设备管理,QQ邮箱,钉钉,脉脉,Microsoft Word,Micr..."
动作冒险,动作冒险,"Dark,汤姆猫跑酷,天天炫斗,大爆炸,滑雪大冒险,崩坏3,登山赛车2,三国战纪,奥特曼传奇..."
娱乐,娱乐,"游戏中心,西瓜视频,掌上英雄联盟,夜景,心悦俱乐部,TapTap,最右,快手直播伴侣,同桌游..."
安全,安全,"云服务,QQ安全中心,隐私空间,系统管家,百度手机卫士,360手机卫士,私密空间,隐私保护,..."


In [354]:
dfcategroup.to_csv('app_cate.csv',sep = '\t',encoding = 'utf-8',index = None )

In [355]:
a = {1,2,3}
b = {2,4,6}
a&b

{2}

In [352]:
dfcategroup 

,app_cate,app_name
app_cate,,
休闲益智,休闲益智,"珊瑚海,媒体存储器,贪吃蛇大作战,球球大作战,节奏大师,迷你世界,我的汤姆猫,腾讯桌球,我的..."
体育竞速,体育竞速,"登山赛车,天天飞车,城市飞车,街头篮球,FIFA足球世界,街机台球大师,疾风飞车世界,汤姆猫..."
健康,健康,"华为穿戴,三星健康,平安好医生,悦动圈,Keep,小米运动,咕咚,健康卫士,乐心运动,优健康..."
儿童,儿童,"儿童模式,宝宝树孕育,小伴龙,儿歌点点,西瓜皮,凯叔讲故事,贝乐虎儿歌,宝宝小警察,宝宝巴士..."
出行,出行,"高德地图,百度地图,驾驶模式,神州专车,位置服务,找回手机,微车违章查询,滴滴顺风车,奥维互..."
办公,办公,"WPS Office,设备管理,QQ邮箱,钉钉,脉脉,Microsoft Word,Micr..."
动作冒险,动作冒险,"Dark,汤姆猫跑酷,天天炫斗,大爆炸,滑雪大冒险,崩坏3,登山赛车2,三国战纪,奥特曼传奇..."
娱乐,娱乐,"游戏中心,西瓜视频,掌上英雄联盟,夜景,心悦俱乐部,TapTap,最右,快手直播伴侣,同桌游..."
安全,安全,"云服务,QQ安全中心,隐私空间,系统管家,百度手机卫士,360手机卫士,私密空间,隐私保护,..."


In [366]:
dfvec = pd.read_csv('app_vec.csv',sep = '\t',encoding = 'utf-8',index_col = 0)

In [371]:
dfvechit = dfvec.loc[{'融360','微信','haha'},:]

In [374]:
app_vec = dfvechit.mean() 

In [375]:
[str(i) for i in app_vec ]

['-0.1215169969946146',
 '0.2263110801577568',
 '-0.11403783038258553',
 '-0.10580507852137089',
 '-0.13848163932561874',
 '0.05902207177132368',
 '0.05116071691736578',
 '-0.009881746955215934',
 '-0.06932300329208374',
 '-0.010298439767211674',
 '0.009753916179761289',
 '0.18767999112606049',
 '-0.06781348027288914',
 '-0.06153331324458122',
 '-0.04013209044933319',
 '0.05536395125091076',
 '0.04585990123450757',
 '0.04752705432474613',
 '-0.08146653696894646',
 '-0.07301262579858303',
 '0.03726204624399543',
 '-0.017123333178460598',
 '-0.0944965835660696',
 '0.010475757764652371',
 '-0.08655692730098963',
 '-0.04632934741675854',
 '0.009341204538941387',
 '-0.07783494889736176',
 '0.04652390629053116',
 '-0.06871942337602377',
 '-0.0407713046297431',
 '-0.07732049003243446',
 '0.1377851739525795',
 '0.14040375128388405',
 '0.049070386216044426',
 '-0.07649123482406141',
 '0.02299698139540851',
 '0.04463003389537334',
 '-0.0025462694466114044',
 '0.03093546722084284',
 '0.0595366302

In [365]:
dfvechit.mean(axis = 0)

0    -0.121517
1     0.226311
2    -0.114038
3    -0.105805
4    -0.138482
5     0.059022
6     0.051161
7    -0.009882
8    -0.069323
9    -0.010298
10    0.009754
11    0.187680
12   -0.067813
13   -0.061533
14   -0.040132
15    0.055364
16    0.045860
17    0.047527
18   -0.081467
19   -0.073013
20    0.037262
21   -0.017123
22   -0.094497
23    0.010476
24   -0.086557
25   -0.046329
26    0.009341
27   -0.077835
28    0.046524
29   -0.068719
        ...   
70    0.106017
71   -0.065686
72    0.008813
73   -0.024000
74   -0.023962
75    0.059404
76    0.105298
77   -0.021284
78    0.002518
79    0.015434
80    0.157682
81    0.057240
82    0.204195
83   -0.123985
84    0.040492
85    0.056879
86    0.024126
87   -0.006140
88    0.083524
89    0.019428
90    0.046018
91    0.016171
92    0.097835
93   -0.250120
94    0.039964
95   -0.177414
96   -0.006075
97    0.034593
98    0.053127
99   -0.011964
Length: 100, dtype: float64

In [ ]:
!pip install 

In [1]:
from hyperopt import fmin, tpe, hp

ModuleNotFoundError: No module named 'hyperopt'

In [ ]:
best = fmin(
    fn=lambda x: x,
    space=hp.uniform('x', 0, 1),
    algo=tpe.suggest,
    max_evals=100)
print(best)